In [ ]:
import horizontal_area as ha
import high_low_xuejie_zuhui as hl
filename = '000001.SZ.csv'
draw_n_days = 400
df = hl.df_init(filename)


In [ ]:
df

In [ ]:
result

In [ ]:
high_points, low_points = hl.find_high_low(df, save_data=False)

In [ ]:
peaks, valleys, high_points, low_points = hl.find_high_low_old(df, save_data=False, draw=True)
result = ha.find_ha_near_hl(df, high_points, low_points)
ha.draw_horizontal_area(df, result, high_points, low_points, filename, n_days=200)

In [ ]:
# 完整测试
import horizontal_area as ha
import high_low_xuejie_zuhui as hl
filename = '000001.SZ.csv'
df = hl.df_init(filename)
high_points, low_points = hl.find_high_low(df, filename, save_data=False, draw_n_days=400)
result = ha.find_horizontal_area(df, high_points, low_points, fft_percentile=100, min_len_of_window=3, ignore_hl=False)
ha.draw_horizontal_area(df, result, high_points, low_points, filename, n_days=200, print_result=False, show_plot=True, save_plot=False)

In [ ]:
ha.draw_horizontal_area(df, result, high_points, low_points, filename, n_days=100, print_result=False, show_plot=True, save_plot=False)

In [ ]:
# 对上方得到的data曲线进行DFT变换，得到频域上的信号
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
import pandas as pd
import high_low_xuejie_zuhui as hl
filename = '000002.SZ.csv'
df = hl.df_init(filename)
data = df.tail(200)
data.plot(x='TRADE_DT', y='S_DQ_CLOSE', figsize=(20, 10))
# 对data进行DFT变换
data_fft = fft(data['S_DQ_CLOSE'].values)
# 可视化
plt.figure(figsize=(20, 10))
plt.plot(data_fft)
plt.show()
# 计算频谱中的总能量，并计算高频成分所占的比例。如果高频成分所占比例较小，那么曲线就比较平滑。
# 计算频谱中的总能量
energy = (data_fft * np.conj(data_fft)).real
# 计算高频成分所占的比例
high_freq_ratio = np.sum(energy[10:]) / np.sum(energy)
print(high_freq_ratio)
# 对高频成分进行滤波
data_fft_filtered = data_fft.copy()
data_fft_filtered[15:] = 0
# 对滤波后的数据进行逆变换
data_filtered = ifft(data_fft_filtered)
# 可视化
plt.figure(figsize=(20, 10))
plt.plot(data_filtered)
plt.show()

In [ ]:
# 对data曲线进行DFT变换，得到频域上的信号
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
import pandas as pd
import high_low_xuejie_zuhui as hl
filename = '000002.SZ.csv'
df = hl.df_init(filename)
data = df.tail(200)
# 将data等分为20份，对每份按照上述方法求的高频成分所占比例
n = 20
data_list = np.array_split(data, n)
high_freq_ratio_list = []
for temp_data in data_list:
    data_fft = fft(temp_data['S_DQ_CLOSE'].values)
    energy = (data_fft * np.conj(data_fft)).real
    high_freq_ratio = np.sum(energy[3:]) / np.sum(energy)
    high_freq_ratio_list.append(high_freq_ratio)
# 可视化
plt.figure(figsize=(20, 10))
plt.plot(high_freq_ratio_list)
plt.show()
print(high_freq_ratio_list)
# 使用plt.axvspan将高频成分占比可视化到data曲线上，高频占比越高的区域颜色越深
plt.figure(figsize=(20, 10))
plt.plot(data['S_DQ_CLOSE'].values)
for i in range(len(high_freq_ratio_list)):
    plt.axvspan(i * 10, (i + 1) * 10, facecolor='r',
                alpha=high_freq_ratio_list[i] * 100)

---

In [ ]:
# hl median测试
filename = '000001.SZ.csv'
df = hl.df_init(filename)
high_points, low_points = hl.find_high_low(
    df, filename, save_data=False, draw=False)
result = ha.find_ha_near_hl_median(df, high_points, low_points, 0.03)
ha.draw_horizontal_area(df, result, high_points, low_points, filename,
                        n_days=270, print_result=False, show_plot=True, save_plot=False)


In [ ]:
high_points, low_points = hl.find_high_low(df, filename, save_data=False, draw_n_days=400)
ha.draw_horizontal_area(df, result, high_points, low_points, filename, n_days=200, print_result=False, show_plot=True, save_plot=False)

In [ ]:
rr = pd.DataFrame(ha.cal_diff_rate_between_medians(result))
rr.columns = ['change_rate']
rr['flag'] = rr['change_rate'].apply(lambda x: '上涨' if x >= 0 else '下跌')
rr['change_rate'] = rr['change_rate']*100
rr

In [ ]:
# 绘制result中位数的boxplot
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif']=['Arial Unicode MS'] 
plt.rcParams['figure.figsize'] = [5, 10]
sns.boxplot(x='flag', y='change_rate', data=rr)
plt.show()
# 绘制histogram
plt.rcParams['figure.figsize'] = [7,5]
sns.histplot(data=rr, x='change_rate', hue='flag', bins=40)


In [ ]:
rr.hist()

In [ ]:
import pandas as pd
import horizontal_area as ha
import high_low_xuejie_zuhui as hl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

count = 0
cr_all = pd.DataFrame()
data_directory = '/Users/kai/Desktop/qs/data_csv_distinct_0606'
for filename in tqdm(os.listdir(data_directory)):
    if filename.endswith('.csv') & (count<100):
        df = hl.df_init(filename)

        high_points, low_points = hl.find_high_low(
            df, filename, save_data=False, draw=False)
        if pd.DataFrame(high_points).empty:
            print(filename, 'no high points')
        if pd.DataFrame(low_points).empty:
            print(filename, 'no low points')
        result = ha.find_ha_near_hl_median(df, high_points, low_points, 0.03)
        # ha.draw_horizontal_area(df, result, high_points, low_points, filename,
        #                         n_days=400, print_result=False, show_plot=True, save_plot=False)
        change_rate = pd.DataFrame(ha.cal_diff_rate_between_medians(result))
        if change_rate.empty:
            continue
        change_rate.columns = ['change_rate']
        change_rate['flag'] = change_rate['change_rate'].apply(lambda x: '上涨' if x >= 0 else '下跌')
        change_rate['change_rate'] = change_rate['change_rate']*100
        cr_all = cr_all.append(change_rate)
        count+=1

# 重置index
cr_all.reset_index(drop=True, inplace=True)
plt.rcParams['figure.figsize'] = [7,5]
sns.histplot(data=cr_all, x='change_rate', hue='flag', bins=40)
plt.show()


In [ ]:
plt.rcParams['figure.figsize'] = [7,5]
# 去除异常值
plt.rcParams['font.sans-serif']=['Arial Unicode MS'] 
cr_all = cr_all[cr_all['change_rate']<200]
sns.histplot(data=cr_all, x='change_rate', hue='flag', bins=60)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [5, 8]
plt.rcParams['font.sans-serif']=['Arial Unicode MS'] 
# 绘制boxplot并在图中每个box中位数上用文字写出数值
sns.boxplot(x='flag', y='change_rate', data=cr_all)
for i in range(len(cr_all.groupby('flag'))):
    plt.text(i, cr_all.groupby('flag')['change_rate'].median()[i], 
             round(cr_all.groupby('flag')['change_rate'].median()[i], 2), 
             ha='center', va='bottom', fontsize=12)
plt.show()

In [ ]:
cr_all['change_rate'] = abs(cr_all['change_rate'])
plt.rcParams['figure.figsize'] = [5, 8]
plt.rcParams['font.sans-serif']=['Arial Unicode MS'] 
# 绘制boxplot并在图中每个box中位数上用文字写出数值
sns.boxplot(x='flag', y='change_rate', data=cr_all)
for i in range(len(cr_all.groupby('flag'))):
    plt.text(i, cr_all.groupby('flag')['change_rate'].median()[i], 
             round(cr_all.groupby('flag')['change_rate'].median()[i], 2), 
             ha='center', va='bottom', fontsize=12)
plt.show()

In [ ]:
# 去除cr_all中的异常值
cr_all = cr_all[cr_all['change_rate']<200]
ha.draw_medians_all(cr_all)

In [ ]:
heal = 0
baoji = 0.51+0.32
baoshang = 0.61
loss = 0.9
a = round(567+0.8*(4655+2394*(1+heal))+(567+0.8*(4655+2394*(1+heal)))*baoji*(baoshang))
final = round(540+0.8*(4655+2394*(1+heal))+0.8*loss*(4655+2394*(1+heal))+(540+0.8*(4655+2394*(1+heal))+0.8*loss*(4655+2394*(1+heal)))*baoji*(baoshang))
print(a, final)

In [ ]:
heal = 0
baoji = 0.51
baoshang = 0.61+0.64
loss = 0.9
a = round(567+0.8*(4655+2394*(1+heal))+(567+0.8*(4655+2394*(1+heal)))*baoji*(baoshang))
final = round(540+0.8*(4655+2394*(1+heal))+0.8*loss*(4655+2394*(1+heal))+(540+0.8*(4655+2394*(1+heal))+0.8*loss*(4655+2394*(1+heal)))*baoji*(baoshang))
print(a, final)

In [1]:
# BEGIN: 9f7a8d3d9d9c
import pandas as pd
from close_position import close_position

def test_close_position():
    # Test case 1: empty dataframe
    df = pd.DataFrame(columns=['date', 'price'])
    try:
        close_position(df)
    except ValueError as e:
        assert str(e) == 'Input df must contain at least 2 rows.'

    # Test case 2: input df is not a DataFrame
    df = 'not a dataframe'
    try:
        close_position(df)
    except TypeError as e:
        assert str(e) == 'Input df must be a DataFrame.'

    # Test case 3: input threshold is not a float
    df = pd.DataFrame({'date': ['2022-01-01', '2022-01-02'], 'price': [100, 109]})
    threshold = 'not a float'
    try:
        close_position(df, threshold)
    except TypeError as e:
        assert str(e) == 'Input threshold must be a float.'

    # Test case 4: input threshold is negative
    threshold = -0.1
    try:
        close_position(df, threshold)
    except ValueError as e:
        assert str(e) == 'Input threshold must be positive.'

    # Test case 5: no close position
    threshold = 0.1
    expected_output = pd.DataFrame(columns=['date', 'price', 'status'])
    assert close_position(df, threshold).equals(expected_output)

    # Test case 6: close position due to stop loss
    df = pd.DataFrame({'date': ['2022-01-01', '2022-01-02', '2022-01-03'], 'price': [100, 90, 80]})
    expected_output = pd.DataFrame({'date': ['2022-01-02'], 'price': [90], 'status': [0]})
    assert close_position(df, threshold).equals(expected_output)

    # Test case 7: close position due to stop profit
    df = pd.DataFrame({'date': ['2022-01-01', '2022-01-02', '2022-01-03'], 'price': [100, 110, 120]})
    expected_output = pd.DataFrame({'date': ['2022-01-02'], 'price': [110], 'status': [1]})
    assert close_position(df, threshold).equals(expected_output)

test_close_position()
# END: 9f7a8d3d9d9c

止盈 2022-01-02 110


110.00000000000001